In [25]:
import pickle
import joblib
import numpy as np
import re

In [1]:
from bpe_tokenizer import BPETokenizer
from tfidf_vectorizer import CustomTfidfVectorizer
from ensemble import EnsembleClassifier

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [38]:
from tokenizers import (
    decoders,
    models,
    normalizers,
    pre_tokenizers,
    processors,
    trainers,
    Tokenizer
)
from transformers import PreTrainedTokenizerFast
from tqdm.auto import tqdm
from datasets import Dataset

In [39]:
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import SGDClassifier

In [2]:
def dummy(text):
    return text
loaded_tokenizer = joblib.load(open('/Users/mahdiislam/Higher Studies/MAIA Study/Semester 1/Software Engineering/Project/Artificial_text_detection/tokenizer.pkl', 'rb'))
loaded_vectorizer = joblib.load(open('/Users/mahdiislam/Higher Studies/MAIA Study/Semester 1/Software Engineering/Project/Artificial_text_detection/tfidf_vectorizer.pkl', 'rb'))
loaded_model = pickle.load(open('/Users/mahdiislam/Higher Studies/MAIA Study/Semester 1/Software Engineering/Project/Artificial_text_detection/trained_model.sav', 'rb'))

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [2]:
alphabets= "([A-Za-z])"
prefixes = "(Mr|St|Mrs|Ms|Dr)[.]"
suffixes = "(Inc|Ltd|Jr|Sr|Co)"
starters = "(Mr|Mrs|Ms|Dr|Prof|Capt|Cpt|Lt|He\s|She\s|It\s|They\s|Their\s|Our\s|We\s|But\s|However\s|That\s|This\s|Wherever)"
acronyms = "([A-Z][.][A-Z][.](?:[A-Z][.])?)"
websites = "[.](com|net|org|io|gov|edu|me)"
digits = "([0-9])"
multiple_dots = r'\.{2,}'

def split_into_sentences(text: str) -> list[str]:
    """
    Split the text into sentences.

    If the text contains substrings "<prd>" or "<stop>", they would lead 
    to incorrect splitting because they are used as markers for splitting.

    :param text: text to be split into sentences
    :type text: str

    :return: list of sentences
    :rtype: list[str]
    """
    text = " " + text + "  "
    text = text.replace("\n"," ")
    text = re.sub(prefixes,"\\1<prd>",text)
    text = re.sub(websites,"<prd>\\1",text)
    text = re.sub(digits + "[.]" + digits,"\\1<prd>\\2",text)
    text = re.sub(multiple_dots, lambda match: "<prd>" * len(match.group(0)) + "<stop>", text)
    if "Ph.D" in text: text = text.replace("Ph.D.","Ph<prd>D<prd>")
    text = re.sub("\s" + alphabets + "[.] "," \\1<prd> ",text)
    text = re.sub(acronyms+" "+starters,"\\1<stop> \\2",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>\\3<prd>",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>",text)
    text = re.sub(" "+suffixes+"[.] "+starters," \\1<stop> \\2",text)
    text = re.sub(" "+suffixes+"[.]"," \\1<prd>",text)
    text = re.sub(" " + alphabets + "[.]"," \\1<prd>",text)
    if "”" in text: text = text.replace(".”","”.")
    if "\"" in text: text = text.replace(".\"","\".")
    if "!" in text: text = text.replace("!\"","\"!")
    if "?" in text: text = text.replace("?\"","\"?")
    text = text.replace(".",".<stop>")
    text = text.replace("?","?<stop>")
    text = text.replace("!","!<stop>")
    text = text.replace("<prd>",".")
    sentences = text.split("<stop>")
    sentences = [s.strip() for s in sentences]
    if sentences and not sentences[-1]: sentences = sentences[:-1]
    return sentences

In [4]:
import re
test = []
text = "Phones\n\nModern humans today are always on their phone. They are always on their phone more than 5 hours a day no stop .All they do is text back and forward and just have group Chats on social media. They even do it while driving. They are some really bad consequences when stuff happens when it comes to a phone. Some certain areas in the United States ban phones from class rooms just because of it.\n\nWhen people have phones, they know about certain apps that they have .Apps like Facebook Twitter Instagram and Snapchat. So like if a friend moves away and you want to be in contact you can still be in contact by posting videos or text messages. People always have different ways how to communicate with a phone. Phones have changed due to our generation.\n\nDriving is one of the way how to get around. People always be on their phones while doing it. Which can cause serious Problems. That's why there's a thing that's called no texting while driving. That's a really important thing to remember. Some people still do it because they think It's stupid. No matter what they do they still have to obey it because that's the only way how did he save.\n\nSometimes on the news there is either an accident or a suicide. It might involve someone not looking where they're going or tweet that someone sent. It either injury or death. If a mysterious number says I'm going to kill you and they know where you live but you don't know the person's contact ,It makes you puzzled and make you start to freak out. Which can end up really badly.\n\nPhones are fine to use and it's also the best way to come over help. If you go through a problem and you can't find help you ,always have a phone there with you. Even though phones are used almost every day as long as you're safe it would come into use if you get into trouble. Make sure you do not be like this phone while you're in the middle of driving. The news always updated when people do something stupid around that involves their phones. The safest way is the best way to stay safe.    "
text2 = "I have always had a strong academic focus and this has led to exceptional results in various exams. My natural curiosity has allowed me to deeply understand various subjects. This was evident in my performance on the entrance exam at Islamic University of Technology (IUT) where I completed my four-year bachelor's degree with a fully funded OIC scholarship, being one of the top 2% of applicants. I discovered my aptitude and interest in the field of machine learning and signal processing in my third year. To deepen my understanding of these fields, I took courses such as Digital Signal Processing, Digital Filter Design, Artificial Neural Networks and Fuzzy Logic during my last two years of undergraduate study hence having a strong foundation in image processing, fourier analysis which are essential aspects in medical image analysis. After acquiring comprehensive knowledge in these domains, I now intend to merge them with the biomedical field by analyzing biomedical images and signals in further research prospects. This interest became solidified while I was researching and fixing an intriguing undergraduate thesis topic."
sentences = split_into_sentences(text2)
# for t in sentences:

#     test.append(loaded_tokenizer.tokenize(t))
# # test.append(loaded_tokenizer.tokenize(text))
# test

In [ ]:
bpe = BPETokenizer(30522).train()


In [33]:
X_test = loaded_vectorizer.transform(test)

In [34]:
proba = loaded_model.predict_proba(X_test)
proba = [inner_list[1] for inner_list in proba]
proba

[0.9795895963043901,
 0.6777920303267865,
 0.47354796028474505,
 0.8117884798348878,
 0.8957399514117084,
 0.7733048239534615,
 0.7728307785962094]

In [35]:
preds = loaded_model.predict(X_test)
preds

array([1, 1, 0, 1, 1, 1, 1])

using this code is it possible to use it in streamlit to host an app where I take a text input and then based on the prediction I will annotate the sentences that are labelled as '1' from the pred variable?